In [4]:
# 1.2
def memory(n):
    """
    >>> f = memory(10)
    >>> f(lambda x: x * 2)
    20
    >>> f(lambda x: x - 7)
    13
    >>> f(lambda x: x > 5)
    True
    """
    def adjust(f):
        nonlocal n
        n=f(n)
        return n
    return adjust 

In [5]:
import doctest
doctest.testmod(verbose = True)

Trying:
    f = memory(10)
Expecting nothing
ok
Trying:
    f(lambda x: x * 2)
Expecting:
    20
ok
Trying:
    f(lambda x: x - 7)
Expecting:
    13
ok
Trying:
    f(lambda x: x > 5)
Expecting:
    True
ok
1 items had no tests:
    __main__
1 items passed all tests:
   4 tests in __main__.memory
4 tests in 2 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)

In [101]:
#1.3
def nonlocalist():
    """
    >>> prepend, get = nonlocalist()
    >>> prepend(2)
    >>> prepend(3)
    >>> prepend(4)
    >>> get(0)
    4
    >>> get(1)
    3
    >>> get(2)
    2
    >>> prepend(8)
    >>> get(2)
    3
    """
    get = lambda x: "Index out of range!"
    def prepend(value):
        nonlocal get
        f = get
        def get(i):
            if i == 0:
                return value
            return f(i-1)
    return prepend, lambda x: get(x)

In [113]:
#2.1
>>> lst = [6, 1, "a"]
>>> next(lst)
# error not iterable

TypeError: 'list' object is not an iterator

In [114]:
>>> lst_iter = iter(lst)
>>> next(lst_iter)
# 6

6

In [115]:
>>> next(lst_iter)
# 1

1

In [116]:
>>> next(iter(lst))
# 6

6

In [117]:
>>> [x for x in lst_iter]
# 'a'

['a']

In [119]:
#2.1
def merge(a, b):
    """
    >>> def sequence(start, step):
    ...     while True:
    ...         yield start
    ...         start += step
    >>> a = sequence(2, 3) # 2, 5, 8, 11, 14, ...
    >>> b = sequence(3, 2) # 3, 5, 7, 9, 11, 13, 15, ...
    >>> result = merge(a, b) # 2, 3, 5, 7, 8, 9, 11, 13, 14, 15
    >>> [next(result) for _ in range(10)]
    [2, 3, 5, 7, 8, 9, 11, 13, 14, 15]
    """
    x=next(a)
    y=next(b)
    while True:   
        if x<y:
            yield x
            x=next(a)
        elif x==y:
            yield x
            x=next(a)
            y=next(b)
        else:
            yield y
            y=next(b)

In [120]:
doctest.testmod(verbose = True)

Trying:
    f = memory(10)
Expecting nothing
ok
Trying:
    f(lambda x: x * 2)
Expecting:
    20
ok
Trying:
    f(lambda x: x - 7)
Expecting:
    13
ok
Trying:
    f(lambda x: x > 5)
Expecting:
    True
ok
Trying:
    def sequence(start, step):
        while True:
            yield start
            start += step
Expecting nothing
ok
Trying:
    a = sequence(2, 3) # 2, 5, 8, 11, 14, ...
Expecting nothing
ok
Trying:
    b = sequence(3, 2) # 3, 5, 7, 9, 11, 13, 15, ...
Expecting nothing
ok
Trying:
    result = merge(a, b) # 2, 3, 5, 7, 8, 9, 11, 13, 14, 15
Expecting nothing
ok
Trying:
    [next(result) for _ in range(10)]
Expecting:
    [2, 3, 5, 7, 8, 9, 11, 13, 14, 15]
ok
Trying:
    prepend, get = nonlocalist()
Expecting nothing
ok
Trying:
    prepend(2)
Expecting nothing
ok
Trying:
    prepend(3)
Expecting nothing
ok
Trying:
    prepend(4)
Expecting nothing
ok
Trying:
    get(0)
Expecting:
    4
ok
Trying:
    get(1)
Expecting:
    3
ok
Trying:
    get(2)
Expecting:
    2
ok
Trying:


TestResults(failed=0, attempted=18)

In [122]:
#2.2
def generate_subsets():
    """
    >>> subsets = generate_subsets()
    >>> for _ in range(3):
    ... print(next(subsets))
    ...
    [[]]
    [[], [1]]
    [[], [1], [2], [1, 2]]
    """
    def subsets(m):
        if m==0:
            return [[]]
        return subsets(m-1)+[n+[m] for n in subsets(m-1)]
    count=0
    while True:
        yield subsets(count)
        count+=1   

In [123]:
subsets = generate_subsets()
for _ in range(3):
    print(next(subsets))

[[]]
[[], [1]]
[[], [1], [2], [1, 2]]


In [131]:
def tree(label, branches=[]):
    for branch in branches:
        assert is_tree(branch)
    return [label] + list(branches)
def label(tree):
    return tree[0]
def branches(tree):
    return tree[1:]
def is_tree(tree):
    if type(tree) != list or len(tree)<1:
        return False
    for branch in branches(tree):
        if not is_tree(branch):
            return False
    return True
def is_leaf(tree):
    return not branches(tree)

In [136]:
#2.3
def sum_paths_gen(t):
    """
    >>> t1 = tree(5)
    >>> next(sum_paths_gen(t1))
    5
    >>> t2 = tree(1, [tree(2, [tree(3), tree(4)]), tree(9)])
    >>> sorted(sum_paths_gen(t2))
    [6, 7, 10]
    """
    if is_leaf(t):
        yield label(t)
    for branch in branches(t):
        for _ in sum_paths_gen(branch):
            yield label(t)+_

In [134]:
t1 = tree(5)
next(sum_paths_gen(t1))

5

In [137]:
t2 = tree(1, [tree(2, [tree(3), tree(4)]), tree(9)])
sorted(sum_paths_gen(t2))

[6, 7, 10]

In [150]:
def collect_words(t):
    """Return a list of all the words contained in the tree where the value of each node in
    the tree is an individual letter. Words terminate at the leaf of a tree.
    >>> collect_words(greetings)
    ['hi', 'hello', 'hey']
    """
    if is_leaf(t):
        return label(t)
    words = []
    for branch in branches(t):
        words += [label(t) + w for w in collect_words(branch)]
    return words

In [151]:
greetings = tree('h', [tree('i'),
                      tree('e', [tree('l', [tree('l', [tree('o')])]),
                                 tree('y')])])

In [152]:
collect_words(greetings)

['hi', 'hello', 'hey']